In [20]:
# Define a pace data structure
class Pace():
    def __init__(self, *args):
        arg_len = len(args)
        if arg_len == 1:
            if isinstance(args[0], str):
                args = args[0].split(":")
                arg_len = len(args)
            else:
                raise ValueError("Pace must be initialized with at least seconds")
        elif arg_len == 2:
            self.minutes = args[0]
            self.seconds = args[1]
            self.tenths = 0
            self.hours = 0
        elif arg_len == 3:
            self.hours = 0
            self.minutes = args[0]
            self.seconds = args[1]
            self.tenths = args[2]
        elif arg_len == 4:
            self.hours = args[0]
            self.minutes = args[1]
            self.seconds = args[2]
            self.tenths = args[3]
        self.dist_unit = "km"

    def __add__(self, other): 
        if self.tenths != 0 or other.tenths != 0:
            self.tenths += other.tenths
            if self.tenths >= 60:
                self.seconds += 1
                self.tenths -= 60
        self.seconds += other.seconds 
        if self.seconds >= 60:
            self.minutes += 1
            self.seconds -= 60
        self.minutes += other.minutes
        if self.minutes >= 60:
            self.hours += 1
            self.minutes -= 60
        if (self.hours != 0) or (other.hours != 0):
            self.hours += other.hours
            
        return Pace(self.hours, self.minutes, self.seconds, self.tenths)
    
    def __sub__(self, other):
        if self.tenths != 0 or other.tenths != 0:
            self.tenths -= other.tenths
            if self.tenths < 0:
                self.seconds -= 1
                self.tenths += 60
        self.seconds -= other.seconds
        if self.seconds < 0:
            self.minutes -= 1
            self.seconds += 60
        self.minutes -= other.minutes
        if self.minutes < 0 and self.hours > 0:
            self.hours -= 1
            self.minutes += 60
        elif self.minutes < 0 and self.hours == 0:
            raise ValueError("Can't have negative pace")
        if (self.hours != 0) or (other.hours != 0):
            if self.hours == 0:
                self.hours = 0
            if other.hours == 0:
                raise ValueError("Can't have negative pace")
        return Pace(self.hours, self.minutes, self.seconds, self.tenths)
    
    def __mul__(self, scalar):
        if scalar < 0:
            raise ValueError("Can't have negative pace")
        self.tenths *= scalar
        self.seconds *= scalar
        self.minutes *= scalar
        self.hours *= scalar
        if self.tenths % 1 > 0:
            self.seconds += round((self.tenths % 1) * 60, 2)
        if self.tenths >= 60:
            self.seconds += 1
            self.tenths -= 60
        if self.seconds % 1 > 0:
            self.tenths = round((self.seconds % 1) * 60, 2)
        if self.seconds >= 60:
            self.minutes += 1
            self.seconds -= 60
        if self.minutes >= 60:
            self.hours += 1
            self.minutes -= 60
        if self.hours % 1 > 0:
            self.minutes = round((self.hours % 1) * 60, 2)
        return Pace( self.hours, self.minutes, self.seconds, self.tenths)
    
    def __str__(self):
        _str = f"{self.minutes:02d}:{self.seconds:02d}"
        _unit = f"min:sec"
        if self.tenths != 0:
            _str += f":{self.tenths:02d}"
            _unit += ":tenths"
        if self.hours != 0:
            _str = f"{self.hours:02d}:" + _str
            _unit = f"hours:" + _unit
        _unit += f"/{self.dist_unit}"
        return _str+" "+_unit
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return (self.hours == other.hours) and (self.minutes == other.minutes) and (self.seconds == other.seconds) and (self.tenths == other.tenths)



In [21]:
# Test Add and Subtract

new_pace = Pace(5, 30) + Pace(5, 45)
assert new_pace == Pace(11, 15), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")

try:
    Pace(5, 30) - Pace(5, 45)
except ValueError as e:
    assert str(e) == "Can't have negative pace"
    print("Got expected error")

new_pace = Pace(5, 30) - Pace(5, 15)
assert new_pace == Pace(0, 15), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")


Got expected result 11:15 min:sec/km
Got expected error
Got expected result 00:15 min:sec/km


In [22]:
# Test Multiply

new_pace = Pace(5, 30) * 2
assert new_pace == Pace(11, 0), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")



Got expected result 11:00 min:sec/km


In [ ]:
class RunPredictor():
    def __init__(self):
        # Establish fun factors
        self.base_pace = Pace(8, 0)